In [5]:
import sys
sys.path.append('/home/ec2-user/SageMaker')
import toniutils as tu

In [9]:
from keras.models import model_from_json

Using TensorFlow backend.


<h2>Load Models</h2>

In [10]:
import boto3
client = boto3.client('s3')
result = client.get_object(Bucket='bme-bucket', Key='models/keras-model-1.json')

In [11]:
model = model_from_json(result["Body"].read())

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
tu.printoni(tu.dirtoni(model), 3)

add                        get_output_mask_at    predict_classes     
add_loss                   get_output_shape_at   predict_generator   
add_metric                 get_updates_for       predict_on_batch    
add_update                 get_weights           predict_proba       
add_weight                 input                 reset_metrics       
assert_input_compatibility input_mask            reset_states        
build                      input_names           run_internal_graph  
built                      input_shape           save                
call                       input_spec            save_weights        
compile                    inputs                set_weights         
compute_mask               layers                state_updates       
compute_output_shape       load_weights          stateful            
count_params               losses                summary             
dtype                      metrics               supports_masking    
evaluate            

In [22]:
tu.printoni(tu.dirtoni(model.weights[0]), 3)

SaveSliceInfo        graph             scatter_nd_sub    
assign               handle            scatter_nd_update 
assign_add           initial_value     scatter_sub       
assign_sub           initialized_value scatter_update    
batch_scatter_update initializer       set_shape         
constraint           is_initialized    shape             
count_up_to          load              sparse_read       
create               name              to_proto          
device               numpy             trainable         
dtype                op                value             
eval                 read_value                          
from_proto           scatter_add                         
get_shape            scatter_nd_add                      


In [31]:
model.layers[0].get_weights()

[array([[-0.4103929 ,  0.01741073, -0.08463517,  0.4012341 ,  0.2294957 ,
         -0.1780038 , -0.31295332, -0.15318403,  0.08876896,  0.00275239,
         -0.36312282,  0.02379653,  0.15153366,  0.37722582, -0.12111342,
         -0.42496505, -0.04997501,  0.34303093, -0.33645892,  0.23648232,
         -0.32641613, -0.21478899],
        [-0.1946102 ,  0.0259957 , -0.4213297 ,  0.25115198, -0.4168209 ,
          0.04586309, -0.06370905,  0.0515157 ,  0.00415909, -0.06211367,
         -0.07197315,  0.15082955,  0.2596978 ,  0.29049885, -0.21252222,
         -0.38601273, -0.10319337,  0.04489741, -0.31893167,  0.19564217,
         -0.14383337, -0.33041602],
        [-0.38670596, -0.26256192,  0.30238962, -0.16036299,  0.21226114,
         -0.34517825, -0.0435386 , -0.3083022 , -0.11216667, -0.34850058,
          0.37063158,  0.00825617, -0.01480088, -0.14823136,  0.04273212,
         -0.03149796, -0.10659587,  0.08716774,  0.02123323,  0.00710037,
         -0.08203658, -0.13514796],
    